<a href="https://colab.research.google.com/github/AndresMontesDeOca/Laboratorio3/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Experiments

## Libraries

In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# from neuralprophet import NeuralProphet
import tensorflow as tf


import warnings
# warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore')

# Ajustar la opción para mostrar más filas
# pd.set_option('display.max_rows', None)

# Si también quieres mostrar más columnas
# pd.set_option('display.max_columns', None)


# Vamos a suprimir la notacion cientifica
pd.set_option("display.float_format", lambda x:"%.2f" %x)


## Carga Datos

In [10]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# # Ventas
id = "158aOjqxaNO8l97yA6VWJkek_15YVLMhs"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('sell-in.txt')
data_ventas = pd.read_csv("sell-in.txt", sep="\t")
data_ventas['periodo'] = pd.to_datetime(data_ventas['periodo'], format='%Y%m')
data = data_ventas.copy()

# # Productos
id = "15JS_k86LS0sgJXma7BOVXWlyNcMwxdhE"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_productos.txt')
data_productos = pd.read_csv("tb_productos.txt", sep="\t")

# # Stocks
id = "15EV-8f_U7onpA1AcTxxXeD-z8yVR4fQu"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_stocks.txt')
data_stocks = pd.read_csv("tb_stocks.txt", sep="\t")
data_stocks['periodo'] = pd.to_datetime(data_stocks['periodo'], format='%Y%m')

# # Productos a predecir
id = "15LjADctFVwjzQFJvfJGFTEdgZx9xCoId"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('productos_a_predecir.txt')
data_productos_a_predecir = pd.read_csv("productos_a_predecir.txt", sep="\t")



## Group Data

In [11]:
import pandas as pd

def group_data(data_all, data_filter):
    # Filtrar el DataFrame 'data_all' para que solo contenga los 'product_id' presentes en 'data_filter'
    data_filtered = data_all[data_all['product_id'].isin(data_filter['product_id'])]

    # Agrupa los datos por 'product_id' y 'periodo', y calcula la suma de 'tn'
    grouped_data = data_filtered.groupby(['product_id', 'periodo']).sum().reset_index()

    # Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
    pivot_data = grouped_data.pivot(index='periodo', columns='product_id', values='tn')

    # Rellena los NaN
    pivot_data = pivot_data.fillna(0)

    # Asegúrate de que los nombres de las columnas sean strings
    pivot_data.columns = pivot_data.columns.astype(str)

    # Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
    pivot_data.columns.name = None

    return pivot_data

data_new = group_data(data, data_productos_a_predecir)

In [12]:
# Filtrar el DataFrame 'data' para que solo contenga los 'product_id' presentes en 'data_productos_a_predecir'
data_filtered = data[data['product_id'].isin(data_productos_a_predecir['product_id'])]


# Agrupa los datos por 'product_id' y 'periodo', y calcula la suma de 'tn'
grouped_data = data_filtered.groupby(['product_id', 'periodo']).sum().reset_index()

# Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
pivot_data = grouped_data.pivot(index='periodo', columns='product_id', values='tn')

# Rellena los NaN
pivot_data = pivot_data.fillna(0)

# Asegúrate de que los nombres de las columnas sean strings
pivot_data.columns = pivot_data.columns.astype(str)

# Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
pivot_data.columns.name = None

data_2019 = pivot_data.loc['2019']

In [13]:
data_2019

,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1275.77,1266.79,964.77,511.34,363.58,578.74,377.67,543.28,465.48,370.76,...,0.03,0.00,0.04,0.04,0.03,0.04,0.00,0.00,0.00,0.00
2019-02-01,1259.09,1043.01,758.33,441.70,409.90,480.00,368.80,476.99,366.73,337.76,...,0.06,0.00,0.05,0.05,0.06,0.06,0.00,0.00,0.00,0.00
2019-03-01,1470.66,1083.63,638.04,619.77,488.21,502.44,576.23,454.57,525.47,522.88,...,0.05,0.38,0.06,0.07,0.06,0.07,0.02,0.02,0.22,0.12
2019-04-01,1647.64,1287.62,565.34,466.71,625.00,835.48,511.55,403.69,391.28,446.72,...,0.03,0.23,0.02,0.02,0.01,0.02,0.36,0.36,0.12,0.10
2019-05-01,1629.78,1034.99,590.13,603.31,897.26,527.69,394.68,486.37,578.49,408.42,...,0.02,0.47,0.02,0.02,0.01,0.02,0.18,0.18,0.24,0.09
2019-06-01,1109.94,928.36,662.39,667.19,876.40,458.04,536.14,567.42,610.40,600.25,...,0.01,0.08,0.01,0.01,0.01,0.01,0.05,0.05,0.06,0.04
2019-07-01,1678.99,1066.45,715.20,521.72,745.75,343.11,573.37,524.05,716.08,463.92,...,0.02,0.03,0.03,0.05,0.04,0.03,0.03,0.06,0.07,0.00
2019-08-01,1261.35,813.78,635.60,482.13,536.67,262.74,307.83,233.01,520.42,199.86,...,0.02,0.13,0.03,0.03,0.02,0.03,0.02,0.01,0.04,0.01
2019-09-01,1660.01,1090.19,967.77,786.17,879.53,409.96,369.75,330.56,558.46,524.95,...,0.01,0.14,0.00,0.01,0.02,0.01,0.02,0.03,0.02,0.02


## Normalize Data

In [14]:
import pandas as pd

def normalize_data(df, normalization="MinMax"):
    """
    Normaliza cada serie de tiempo (columna) de manera individual usando MinMax o Zscore.

    Args:
        df (pd.DataFrame): DataFrame con series de tiempo de distintos productos, cada columna es un producto.
        normalization (str): Tipo de normalización a aplicar. Opciones: "MinMax" o "Zscore". Default es "MinMax".

    Returns:
        normalized_df (pd.DataFrame): DataFrame con las series normalizadas.
        normalization_params (dict): Diccionario con los parámetros necesarios para desnormalizar cada columna.
            - Para "MinMax": valores min y max de cada columna.
            - Para "Zscore": valores mean y std de cada columna.
    """
    normalization_params = {}
    normalized_df = pd.DataFrame()

    for column in df.columns:
        if normalization == "MinMax":
            min_value = df[column].min()
            max_value = df[column].max()
            normalization_params[column] = {"min": min_value, "max": max_value}
            normalized_df[column] = (df[column] - min_value) / (max_value - min_value)

        elif normalization == "ZScore":
            mean_value = df[column].mean()
            std_value = df[column].std()
            normalization_params[column] = {"mean": mean_value, "std": std_value}
            normalized_df[column] = (df[column] - mean_value) / std_value

        else:
            raise ValueError("Invalid normalization method. Choose 'MinMax' or 'Zscore'.")

    return normalized_df, normalization_params


In [15]:
data

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,2017-01-01,10234,20524,0,2,0.05,0.05
1,2017-01-01,10032,20524,0,1,0.14,0.14
2,2017-01-01,10217,20524,0,1,0.03,0.03
3,2017-01-01,10125,20524,0,1,0.02,0.02
4,2017-01-01,10012,20524,0,11,1.54,1.54
...,...,...,...,...,...,...,...
2945813,2019-12-01,10105,20853,0,1,0.02,0.02
2945814,2019-12-01,10092,20853,0,1,0.01,0.01
2945815,2019-12-01,10006,20853,0,7,0.03,0.03
2945816,2019-12-01,10018,20853,0,4,0.02,0.02


## Split Data

In [16]:
def split_data_all(data):
  data_train = data.loc['2017':'2018']
  data_valid = data.loc['2019']
  return data_train, data_valid

## Window Data

In [22]:
def window_dataset(sequence, data_split, window_size, batch_size, n_future, shuffle_buffer=1000, seed=None):
    """Generates dataset windows for multi-step forecasting in a multivariable context.

    Args:
      sequence (array-like): Contains the values of the time series, where each element is an array of feature values.
      data_split (str): Specifies if the dataset is for training or validation/test.
      window_size (int): The number of time steps to include in the feature.
      batch_size (int): The batch size.
      n_future (int): The number of future steps to predict.
      shuffle_buffer (int): Buffer size to use for the shuffle method.
      seed (int, optional): Random seed for reproducibility.

    Returns:
      tf.data.Dataset: TF Dataset containing time windows.
    """

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(sequence)

    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + n_future, shift=1, drop_remainder=True)

    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + n_future))

    # Create tuples with features and labels
    dataset = dataset.map(lambda window: (window[:window_size], window[window_size:]))

    if data_split == 'train':
        # Shuffle the training data to improve generalization
        dataset = dataset.shuffle(shuffle_buffer, seed=seed)
    else:
        # Cache the validation/test data for improved performance
        dataset = dataset.cache()

    # Create batches of windows and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


## Callbacks

In [27]:
#############################################################################
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

class MAEThresholdCallback(Callback):
    def __init__(self, threshold=0.15):
        super(MAEThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_mae = logs.get('val_mae')
        if val_mae is not None and val_mae <= self.threshold:
            print(f'\nEpoch {epoch+1}: Validation MAE has reached {val_mae:.4f}, stopping training.')
            self.model.stop_training = True

def MyCallbacks(model_name, patience):
    earlystop = EarlyStopping('val_loss', patience=patience, restore_best_weights=True)
    checkpoint = ModelCheckpoint(filepath=f'ckpts/{model_name}-' + '{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss')
    mae_threshold_callback = MAEThresholdCallback(threshold=0.015)
    return [earlystop, checkpoint]#, mae_threshold_callback]

#############################################################################

## Model Design

In [ ]:
#############################################################################
def compile_model(new_model, loss):
  new_model.compile(optimizer='adam', loss=loss, metrics=['mae']) # metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
  print(new_model.summary())
  return new_model
#############################################################################
def MyModel(loss, window_size, n_future, n_features):
    new_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer((window_size, n_features)),
        # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)), # probar esta layer
        # tf.keras.layers.experimental.preprocessing.Resizing(125, 125),
        tf.keras.layers.Conv1D(16, window_size + n_future, activation='relu', padding='causal'),
        tf.keras.layers.MaxPool1D(),
        # tf.keras.layers.LSTM(48, return_sequences=True),
        # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        # tf.keras.layers.LSTM(48, return_sequences=False),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(n_features * n_future, activation='relu'),
        # tf.keras.layers.Dropout(0.4),
        # tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Reshape((n_future, n_features)),
        # tf.keras.layers.Dense(1)
        ])
    return compile_model(new_model, loss)

# Pipeline

## Data Preprocessing

In [28]:
# data
# data_productos
# data_stocks
# data_productos_a_predecir
# # if __name__ == '__main__':
window_size = 12
n_future = 2
batch_size = 64
normalization = 'MinMax'

data_all = group_data(data, data_productos_a_predecir)
data_all_norm, data_all_norm_params = normalize_data(data_all, normalization=normalization)
# data_all_norm['20001'].describe()
data_train, data_valid = split_data_all(data_all_norm)
# print(data_train.shape)
# print(data_valid.shape)
data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)



## Model Train

In [ ]:
# # # #########################################################################
# # # # Neural Netowrk Model
# # #########################################################################
model_name = 'TimeSeries'
loss = 'mse'
patience = 10
epochs = 500
n_features = data_all.shape[1]

callbacks = MyCallbacks(model_name, patience)
# model = MyModel(loss)
model = MyModel(loss, window_size, n_future, n_features)

history = model.fit(
    data_train_windowed,
    validation_data = data_valid_windowed,
    # callbacks = callbacks,
    epochs=epochs)

# plot_history(history, 4)
# save_model(model, model_name, history, data_test_wrangled)
# show_predictions(model, data_test_wrangled, data_test[n_past:])